### Run this in Python kernel

In [ ]:
%%html
<style>
body {
    font-family: "Arial", cursive, sans-serif; # font style of application
}
pre {
    font-family: "Courier New"; # font style of cell outputs
}
</style>  

### Run below in the backtesting_clojure kernel

In [1]:
; import libraries from kernel
(ns clojure-backtesting.demo
  (:require [clojure.test :refer :all]
            [clojure-backtesting.data :refer :all]
            [clojure-backtesting.data-management :refer :all]
            [clojure-backtesting.order :refer :all]
            [clojure-backtesting.evaluate :refer :all]
            [clojure-backtesting.plot :refer :all]
            [clojure-backtesting.counter :refer :all]
            ;;[clojure-backtesting.parameters :refer :all]
            [clojure.string :as str]
            [clojure.pprint :as pprint]
            [java-time :as t]
            [clojupyter.kernel.version :as ver]
            [clojure.edn :as edn]
            [clojupyter.misc.helper :as helper]
  ) ;; require all libriaries from core
  (:use clojure.pprint)
)

nil

### Import dataset

In [2]:
; path to dataset = "../resources/CRSP-extract.csv"
; change it to the relative to your own dataset
;
(reset! data-set (add-aprc (read-csv-row "../resources/CRSP-extract.csv")));

### Get list of tickers with ROE

In [6]:
;; Use merged database CRSP + COMPUSTAT
;; Compute the latest quarter ROE by using: niq/(share price * cshoq) in the merged dataset
;; output {{:tic "AAPL" :year "1980" :ROE x.x}{...}{:ROE 1.x 2.x 3.x 10.x}}

;; need to use lazy sequence if applying on big dataset
(defn get-set-roe
    "return a set of maps of tickers and datadate" ;;{:tic "AAPL", :datadate "1981/3/31", :ROE x.x}
    [file date]
    (loop [remaining file
            result-set []
            ROE-list [] ]
        (if (empty? remaining)
            ;; how to change the output here?
            ;;(into #{} (conj result-set {:ROE-set ROE-list}))
            {:data result-set :ROE-set ROE-list}
            (let [first-line (first remaining)
                next-remaining (rest remaining)
                ;;next-result-set (conj result-set (get first-line :datadate)
                [year month day] (map parse-int (str/split date #"-"))]
                ;;merged file use :datadate as key
                (if (= date (get first-line :datadate))
                    (let [[niq PRC cshoq] (map edn/read-string [(get first-line :niq)(get first-line :PRC)(get first-line :cshoq)])
                          ROE (/ niq (* PRC cshoq))]
                        (recur next-remaining (conj result-set {:tic (get first-line :tic) :year year :ROE ROE}) (conj ROE-list ROE))
                    )
                    (recur next-remaining result-set ROE-list)
                )  
            )
        )
    )
)

#'clojure-backtesting.demo/get-set-roe

In [7]:
;; this function can be used to get the list of stocks available for trading
(def dataFA (get-set-roe (read-csv-row "../resources/data-testing-merged.csv") "1980-12-18"))
(println dataFA) ;

{:data [{:tic AAPL, :year 1980, :ROE 0.002731506525434953} {:tic GM, :year 1980, :ROE -0.046370846428037134} {:tic IBM, :year 1980, :ROE 0.0238072629827958}], :ROE-set [0.002731506525434953 -0.046370846428037134 0.0238072629827958]}


In [8]:
;; fn to get a set of ROE
(defn get-ROE
            "return a set of ROE"  ;;{10.2 1.8 x.x ...}
        [dataset]
        (into #{} (get dataset :ROE-set))
        )

#'clojure-backtesting.demo/get-ROE

In [9]:
(defn get-roe-20
    "return a set tickers" ;;{"AAPL" "GM"}
    [data roe-20]
    (loop [remaining (get data :data)
            result-set []]
        (if (empty? remaining)
            (into #{} result-set)
            (let [first-line (first remaining)
                next-remaining (rest remaining)]
                ;;conj the stock ticker whose ROE >= roe-20
                ;;(if (not= -1 (compare roe-20 (get first-line :ROE)))
                (if (>= (get first-line :ROE) roe-20)
                    (recur next-remaining (conj result-set (get first-line :tic)))           
                    (recur next-remaining result-set)            
                )  
            )
        )
    )
)

#'clojure-backtesting.demo/get-roe-20

In [10]:
;; test examples

;; dataFA is the dataset that contains the output from get-set

;; get a set of ROE
(def roe-list-demo (get-ROE dataFA))
(println roe-list-demo)


;; get a sorted set of ROE
(def roe-sorted-set (apply sorted-set roe-list-demo))
(println roe-sorted-set)


;; get a sorted vector of ROE
(def roe-sorted-vec (into '[] roe-sorted-set))
(println roe-sorted-vec)

;; find the 20th per centile ROE
(def roe-20-demo (nth roe-sorted-vec (int (* 0.8 (count roe-sorted-vec)))))
(println roe-20-demo)
         
;; get the list of tickers to buy
(def stock-to-buy-test (get-roe-20 dataFA roe-20-demo))
(println stock-to-buy-test)
(println (type stock-to-buy-test)) ;;stored in a set format

#{0.0238072629827958 -0.046370846428037134 0.002731506525434953}
#{-0.046370846428037134 0.002731506525434953 0.0238072629827958}
[-0.046370846428037134 0.002731506525434953 0.0238072629827958]
0.0238072629827958
#{IBM}
clojure.lang.PersistentHashSet


nil

### Initialise portfolio

In [11]:
(keys (deref available-tics-))

Execution error (ClassCastException) at clojure-backtesting.demo/eval4336 (REPL:1).
clojure.lang.Var$Unbound cannot be cast to java.base/java.util.concurrent.Future


class java.lang.ClassCastException: 

In [79]:
;; initialise with current date and initial capital (= $1,000,000)
(init-portfolio "1980-12-18" 100000)

(def start-date (get-date))
;; time span = n days
(def time-span 100) 

;; get stock tickers and ROE data
;; output {{:tic "AAPL" :year "1980" :ROE x.x}{...}{:ROE 1.x 2.x 3.x 10.x}}
(def stock-data (get-set-roe (read-csv-row "../resources/data-testing-merged.csv") "1980-12-18"))

;; 1. sort the stocks according to their ROE (= Net Income/Total Equity)

;; get a set of ROE
(def roe-list (get-ROE stock-data))

;; sorting function, return tickers of the top 20% stocks
;; determine what are the top 20% stocks and turn the sorted-set into list format
(def roe-sorted (into '[] (apply sorted-set roe-list)))

;(println roe-list)
;(println (int (* 0.8 (count roe-sorted))))

;;find the 20% cut-off ROE value
(let [roe-20 (nth roe-sorted (int (* 0.8 (count roe-sorted))))]    
  ;;get the tickers of the top 20 with function get-roe-20
  (def stocks-to-buy (get-roe-20 stock-data roe-20))
)

;; 2. buy the top 20% stocks and the sell the stocks that are not in the top 20% this year      
(def stocks-to-buy-list (into [] stocks-to-buy))
;(println stocks-to-buy-list)

(doseq [stock stocks-to-buy-list]        
(order stock 10)
)


(while (not= time-span 0)
  ;; check whether the start day is in the dataset
  ;; find the quarterly date before inputting into get-set function
  (do 
      (next-date) ; move on to the next trading day
      (update-eval-report (get-date)) ; update the eval-report
      (def time-span (dec time-span)) ; decrement time span
  )
)

;; 3. sell all stocks by the end of time span
(doseq [stock stocks-to-buy-list]        
    (order stock -10)
)

nil

### Check order record

In [80]:
(pprint/print-table (deref order-record))


|      :date | :tic | :price | :quantity | :count |
|------------+------+--------+-----------+--------|
| 1980-12-19 |  IBM | 64.625 |        10 |        |
| 1981-05-14 |  IBM | 56.625 |       -10 |        |


nil

### Check portfolio record

In [81]:
;; view final portfolio
(view-portfolio)


| :asset | :price |  :aprc | :quantity | :tot-val |
|--------+--------+--------+-----------+----------|
|   cash |    N/A |    N/A |       N/A |    99920 |
|    IBM | 56.625 | 703.59 |         0 |        0 |


nil

In [82]:
;; view portfolio value and return
(view-portfolio-record)


|      :date | :tot-value | :daily-ret | :tot-ret | :loan | :leverage |
|------------+------------+------------+----------+-------+-----------|
| 1980-12-18 |    $100000 |      0.00% |    0.00% | $0.00 |     0.00% |
| 1980-12-19 |    $106714 |      6.50% |    6.50% | $0.00 |     0.00% |
| 1981-05-14 |     $99920 |     -6.58% |   -0.08% | $0.00 |     0.00% |


nil

### Generate evaluation report

In [65]:
(eval-report)


|      :date | :pnl-pt | :sharpe | :tot-val |  :vol |
|------------+---------+---------+----------+-------|
| 1980-12-19 |   $2278 |   1.67% |  $106836 | 3.95% |
| 1980-12-22 |   $1368 |   1.91% |  $106842 | 3.47% |
| 1980-12-23 |    $986 |   2.13% |  $106903 | 3.13% |
| 1980-12-24 |    $769 |   2.33% |  $106926 | 2.87% |
| 1980-12-26 |    $625 |   2.49% |  $106879 | 2.67% |
| 1980-12-29 |    $531 |   2.67% |  $106908 | 2.50% |
| 1980-12-30 |    $458 |   2.81% |  $106873 | 2.36% |
| 1980-12-31 |    $407 |   2.99% |  $106932 | 2.24% |
| 1981-01-02 |    $367 |   3.15% |  $106986 | 2.14% |
| 1981-01-05 |    $335 |   3.32% |  $107044 | 2.05% |
| 1981-01-06 |    $302 |   3.41% |  $106950 | 1.97% |
| 1981-01-07 |    $274 |   3.49% |  $106860 | 1.90% |
| 1981-01-08 |    $253 |   3.60% |  $106848 | 1.84% |
| 1981-01-09 |    $235 |   3.70% |  $106818 | 1.78% |
| 1981-01-12 |    $219 |   3.81% |  $106806 | 1.73% |
| 1981-01-13 |    $206 |   3.93% |  $106818 | 1.68% |
| 1981-01-14 |    $193 |   

nil

### Plot variables
Below are example codes that show how to plot different variables in the portfolio record / evaluation record.

### 1. Portfolio total value

In [66]:
(def data (deref portfolio-value))

#'clojure-backtesting.demo/data

In [67]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "total value")
  data))

#'clojure-backtesting.demo/data-to-plot

In [68]:
(first data-to-plot)

{:date "1980-12-18", :tot-value 100000, :daily-ret 0.0, :tot-ret 0.0, :loan 0.0, :leverage 0.0, :plot "total value"}

In [69]:
(plot data-to-plot :plot :date :tot-value false)

### 2. Plot sharpe ratio

In [72]:
(def data (deref eval-record))

; Add rename legend
(def data-to-plot
 (map #(assoc % :plot "sharpe ratio")
  data))

(plot data-to-plot :plot :date :sharpe false)

### 3. Plot stock price

In [76]:
(def data (deref order-record))

#'clojure-backtesting.demo/data

In [77]:
(first data)

{:date "1980-12-19", :tic "IBM", :price 64.625, :quantity 10, :count nil}

In [78]:
; Add rename legend
(def data-to-plot
 (map #(assoc % :plot "price")
  data))

(plot data-to-plot :plot :date :price false)